# Install the charting library & tables

In [ ]:
!pip install plotly
!pip install tabulate

## Login 

In [ ]:
import requests
import os
from dotenv import load_dotenv
from google.colab import userdata

# Define the API endpoint and payload
base_url = "https://api-nwfp.rothamsted.ac.uk/api"
endpoint = "auth/v1/login/"
url = f"{base_url}/{endpoint}"

load_dotenv()

payload = {
    #"email": os.getenv("API_EMAIL"),
    #"password": os.getenv("API_PASSWORD")
    "email": userdata.get("API_EMAIL"),
    "password": userdata.get("API_PASSWORD")
}
# Make the POST request
response = requests.post(url, data=payload)

# Check the response
if response.status_code == 200:
    response_data = response.json()
    print("Login successful!")
    token = response_data.get("access")
    headers = {
        "Authorization":f"Bearer {token}",
        "Cache-Control": "no-cache"
    }
    key = token
    print(key)
else:
    print("Failed to login. Status code:", response.status_code)
    print("Response:", response.text)

## Getting help about measurements

The NWFP APIs can provide help information, for example if you want to know the available measurements we can query using the *Help* parameter. This will tell us what the avalable measurement data categories are.

In [ ]:
import requests
import pandas as pd
from tabulate import tabulate

# Define the API endpoint and parameters
endpoint = "measurements/v2/MeasurementsByTypeID/"
api_url = f"{base_url}/{endpoint}"
params = {
    "Help": "measurements"
}
# Make the GET request
response = requests.get(api_url, headers=headers, params=params)

# Check the response
if response.status_code == 200:    
    data = response.json()
    df = pd.DataFrame(data)
    # Reformat the output in a nice table
    print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))
else:
    print("Failed. Status code:", response.status_code)
    print("Response:", response.text)

We can do the same for other parameters, for example, fields

In [ ]:
import requests
import pandas as pd
from tabulate import tabulate

# Define the API endpoint and parameters
endpoint = "measurements/v2/MeasurementsByTypeID/"
api_url = f"{base_url}/{endpoint}"
params = {
    "Help": "field"
}
# Make the GET request
response = requests.get(api_url, headers=headers, params=params)

# Check the response
if response.status_code == 200:    
    data = response.json()
    df = pd.DataFrame(data)
    # Reformat the output in a nice table
    print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))
else:
    print("Failed. Status code:", response.status_code)
    print("Response:", response.text)

## Querying data for specific catchments
In this example, we will query specific measurements for two catchment IDs within a specified date range, we will then visualise the *Flow* data.

In [ ]:

import json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

endpoint = "measurements/v2/MeasurementsByTypeID/"
api_url = f"{base_url}/{endpoint}"
params = {
    "catchmentids": "5,6",
    "measurements": "2,19,20",
    "datefrom": "2022-12-11",
    "dateto": "2023-12-13",
}
response = requests.get(api_url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    df = pd.DataFrame(data)
    
    # Filter data for Flow in Catchment 5 and Catchment 6
    measurement_type = "Flow"
    #measurement_type = "Soil Moisture @ 10cm Depth" 
    df_catchment5 = df.loc[(df['catchmentid'] == 5) & (df['meastypedisplayname'] == measurement_type)].copy()
    df_catchment6 = df.loc[(df['catchmentid'] == 6) & (df['meastypedisplayname'] == measurement_type)].copy()
    
    # Convert datetime strings to datetime objects
    df_catchment5['datetime'] = pd.to_datetime( df_catchment5['datetime'])
    df_catchment6['datetime'] = pd.to_datetime( df_catchment6['datetime'])
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df_catchment5['datetime'], y=df_catchment5['value'],
                        mode='lines', name='Catchment 5', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=df_catchment6['datetime'], y=df_catchment6['value'],
                         mode='lines', name='Catchment 6', line=dict(color='red')))

    fig.update_layout(
            title=f'{measurement_type} Data for Catchments',
            xaxis_title='datetime',
            yaxis_title='value'
        )

    # Show the chart
    fig.show()
else:
    print("Failed. Status code:", response.status_code)
    print("Response:", response.text)


In this example we visualise the soil temperature data

In [ ]:
# Filter data for Flow in Catchment 5 and Catchment 6
measurement_type = "Soil Temperature @ 15cm Depth"   # "Soil Moisture @ 15cm Depth"  or Flow
df_catchment5 = df.loc[(df['catchmentid'] == 5) & (df['meastypedisplayname'] == measurement_type)].copy()
df_catchment6 = df.loc[(df['catchmentid'] == 6) & (df['meastypedisplayname'] == measurement_type)].copy()
    
# Convert datetime strings to datetime objects
df_catchment5['datetime'] = pd.to_datetime( df_catchment5['datetime'])
df_catchment6['datetime'] = pd.to_datetime( df_catchment6['datetime'])
    
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_catchment5['datetime'], y=df_catchment5['value'],
                mode='lines', name='Catchment 5', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df_catchment6['datetime'], y=df_catchment6['value'],
                 mode='lines', name='Catchment 6', line=dict(color='red')))

fig.update_layout(
    title=f'{measurement_type} Data for Catchments',
    xaxis_title='datetime',
    yaxis_title='value'
)

# Show the chart
fig.show()